## References & starter code

https://github.com/dmlc/xgboost/blob/master/demo/guide-python/basic_walkthrough.py

In [1]:
import numpy as np

import xgboost as xgb

import pandas as pd
from tqdm import tqdm

from scipy import sparse
from scipy import stats
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder 

In [2]:
xgb.__version__

'0.90'

In [3]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
import glob

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()
import pickle

In [18]:
!ls /data/work/shajikk/0308/data/full_cleaned

_common_metadata  part.24.parquet  part.41.parquet  part.59.parquet
_metadata	  part.25.parquet  part.42.parquet  part.6.parquet
part.0.parquet	  part.26.parquet  part.43.parquet  part.60.parquet
part.1.parquet	  part.27.parquet  part.44.parquet  part.61.parquet
part.10.parquet   part.28.parquet  part.45.parquet  part.62.parquet
part.11.parquet   part.29.parquet  part.46.parquet  part.63.parquet
part.12.parquet   part.3.parquet   part.47.parquet  part.64.parquet
part.13.parquet   part.30.parquet  part.48.parquet  part.65.parquet
part.14.parquet   part.31.parquet  part.49.parquet  part.66.parquet
part.15.parquet   part.32.parquet  part.5.parquet   part.67.parquet
part.16.parquet   part.33.parquet  part.50.parquet  part.68.parquet
part.17.parquet   part.34.parquet  part.51.parquet  part.69.parquet
part.18.parquet   part.35.parquet  part.52.parquet  part.7.parquet
part.19.parquet   part.36.parquet  part.53.parquet  part.70.parquet
part.2.parquet	  part.37.parquet  part.54.parquet  part.71

In [5]:
result_dd = dd.read_parquet('/data/work/shajikk/0308/data/full_cleaned')

In [6]:
result_dd.head(2)

[########################################] | 100% Completed |  2.0s


,label,text
index,,
0,090111,1 x 20 container described as 275 bags green c...
1,090111,arabica green coffee beans


In [10]:
# files1 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_1*.parq')
# files2 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_2*.parq')
# files3 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_3*.parq')
# files4 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_4*.parq')
# files5 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_5*.parq')
# files6 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_6*.parq')
# files7 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_7*.parq')
# files8 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_8*.parq')
# files9 = glob.glob('/data/common/trade_data/2019_updated/data_samples_ignore_multiple_hscode/sample_by_chapter/sample_chap_9*.parq')

# all_files = files1 + files2 + files3 + files4 + files5 + files6 + files7 + files8 + files9
# result_dd = dd.concat([dd.read_parquet(fp) for fp in all_files])

In [12]:
# df1 = result_dd[['Product Desc', 'Cleaned_HS_Code']]
# df1.columns = ['Desc', 'HSCode']

# print(df1.head(2))
# print(len(df1))

[########################################] | 100% Completed | 19.9s
                                                Desc  HSCode
0  SUPREME ALL PURPOSE LARD IN PAILS 37LBS CAED N...  150110
1  SLAC: 840 PAILS NABORI ALL PURPOSE LARD HS COD...  150110
[########################################] | 100% Completed | 48.1s
1982138


In [14]:
# hs_code_desc = result_dd[['Cleaned_HS_Code', 'Merged_Description']]
# hs_code_desc = hs_code_desc.drop_duplicates()
# hs_code_desc.columns = ['HSCode', 'Desc']
# print(hs_code_desc.head(2))
# print(len(hs_code_desc))

[########################################] | 100% Completed | 46.0s
   HSCode                                               Desc
0  150110  Pig fat (including lard) and poultry fat, othe...
3  150120  Pig fat (including lard) and poultry fat, othe...
[########################################] | 100% Completed | 49.5s
4137


In [23]:
result_dd.dtypes

label    object
text     object
dtype: object

In [7]:
# result_dd = result_dd.append(hs_code_desc[['Desc', 'HSCode']]).reset_index()
result_df = result_dd.compute()
result_df.label = result_df.label.astype(str).str[:2]
result_dd = dd.from_pandas(result_df, npartitions=3)
result_dd.head(2)

[########################################] | 100% Completed |  1.4s
[########################################] | 100% Completed |  0.1s


,label,text
index,,
0,09,1 x 20 container described as 275 bags green c...
0,78,refined lead hs code: xxxx


In [8]:
all_labels = result_dd.label.unique()
print('Number of classes : ', len(all_labels))

[########################################] | 100% Completed |  0.2s
Number of classes :  72


In [9]:
len(result_dd)

[########################################] | 100% Completed |  0.1s


1982138

In [10]:
result = result_dd.compute()

[########################################] | 100% Completed |  0.1s


## Lemmatizer

In [11]:
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

from nltk.tokenize import RegexpTokenizer
tok = RegexpTokenizer(r'\w+')
    
def clean_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    sentence = re.sub(r'\d+', '', sentence)
    remove_dig_pun = tok.tokenize(sentence.lower())

    nltk_tagged = nltk.pos_tag(remove_dig_pun)  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
            
    lemmatized_sentence_clean = list(map((lambda x : x if x not in stop else ""), lemmatized_sentence))
    input_clean = list(map((lambda x : x if x not in stop else ""), remove_dig_pun))

    return " ".join(input_clean + lemmatized_sentence_clean)

## Read the HTS descriptions, process

In [12]:
hts = pd.read_csv("/data/work/shajikk/0308/hts_train.csv", dtype={'hs': str, 'desc' : str})

In [13]:
hts['clean'] = hts.desc.apply(lambda x : clean_sentence(x))
hts = hts.rename({'hs' : 'label', 'clean' : 'text'}, axis=1)[['label', 'text']]

In [14]:

hts.label = hts.label.astype(str).str[:2]
# hts = dd.from_pandas(hts_df, npartitions=3)
# result_dd.head(2)
hts.head(2)

,label,text
0,01,live horses asses mules hinnies horses purebr...
1,01,live horses asses mules hinnies horses import...


## Read the NACIS->HTS examples, process

In [15]:
nacis = pd.read_csv("/data/work/shajikk/0308/commodity_hts_extract.csv", dtype={'hts6': str, 'description_long' : str})

In [16]:
nacis['clean'] = nacis.description_long.apply(lambda x : clean_sentence(x))

In [17]:
nacis = nacis.rename({'hts6' : 'label', 'clean' : 'text'}, axis=1)[['label', 'text']]

In [18]:
nacis.head(1)

,label,text
0,910211,batteries wrist watches battery powered mech...


## Sample the full data, 4137 classes. Create a subset of 100 examples each per class. Save it off

In [104]:
all_train_df = []
all_valid_df = []
count = 5000
recompute = True

if not recompute :
    with open('all_train_df_5k_hschap.pkl', 'rb') as f: all_train_df = pickle.load(f)
    with open('all_valid_df_5k_hschap.pkl', 'rb') as f: all_valid_df = pickle.load(f)
    pass


for c in tqdm(all_labels) :
    if (not recompute) : break
    df = result[result.label == c]
    df_sampled = df.sample(frac=min(count/len(df), 1))
    df_hts = hts[hts.label == c]
    df_nacis  = nacis[nacis.label == c]
    
    
    train_df  = df_sampled.sample(frac=0.8)
    valid_df = df_sampled.drop(train_df.index)

    all_train_df.append(train_df)
    all_train_df.append(df_hts)
    all_train_df.append(df_nacis)
    
    all_valid_df.append(valid_df)

train_df  = pd.concat(all_train_df)
valid_df  = pd.concat(all_valid_df)

if recompute :
    with open('all_train_df_5k_hschap.pkl', 'wb') as f: pickle.dump(all_train_df, f)
    with open('all_valid_df_5k_hschap.pkl', 'wb') as f: pickle.dump(all_valid_df, f)
    pass

[########################################] | 100% Completed |  0.2s


  0%|          | 0/72 [00:00<?, ?it/s]

[########################################] | 100% Completed |  0.2s


In [105]:
len(valid_df['text']), len(train_df['text']), len(train_df['text'])/40000

(67948, 276077, 6.901925)

## Construct count vectorizer with HTS, NACIS keywords

In [106]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vector = CountVectorizer(max_features=30000)
count_vector.fit(list(hts['text']) + list(nacis['text']))

CountVectorizer(max_features=30000)

In [107]:
test = count_vector.transform(list(hts['text']) + list(nacis['text']))
test.shape

(34707, 14588)

In [108]:
X_train_counts = count_vector.transform(list(train_df['text']))
X_train_counts.shape

(276077, 14588)

In [109]:
label_enc = LabelEncoder() 
label_enc.fit(result['label']) 
y_train = np.expand_dims(np.array(label_enc.transform(train_df['label'])), 1)

In [110]:
X_train_counts.shape, y_train.shape

((276077, 14588), (276077, 1))

In [111]:
pickle.dump(count_vector, open("count_vector.pickle", "wb"))

## Batch logic for GBT

In [115]:
def batch(csr, y, rows, random_row_array, n=1):
    l = len(rows)
    for ndx in range(0, l, n):
        yield (csr[random_row_array[ndx:min(ndx + n, l)]].todense(), 
               y[random_row_array[ndx:min(ndx + n, l)]])

## Enable batching class

In [116]:
class make_model():
    def __init__(self, param, lr, num_round = 5, batch_size=1000):
        self.param     = param
        self.num_round = num_round
        self.batch_size = batch_size
        self.lr = lr
        
    def fit(self, csr, y_val):
        iteration = 0
        print("Will run for {} rounds".format(self.num_round))
        for n_round in range(0, self.num_round):     
            random_row_array = np.random.choice(np.arange(csr.shape[0]), csr.shape[0], replace=False)
            rows = range(0, csr.shape[0])
            with tqdm(total=int(len(rows)/self.batch_size)) as progress_bar:
                for x,y in batch(csr, y_val, rows, random_row_array, self.batch_size):
                    dtrain = xgb.DMatrix(x, y)
                    watchlist = [(dtrain,'train')]

                    if iteration == 0 : model = xgb.Booster(self.param, [dtrain])
                    
                    self.param['eta'] = self.lr[iteration]
                    print('Round = {}, Iteration = {}, lr = {}'.format(n_round, iteration, self.lr[iteration]))
                    
                    model = xgb.train(self.param, dtrain, num_boost_round=1, xgb_model=model, evals=watchlist)
                    iteration = iteration + 1
                    progress_bar.update(1)
            if n_round > 4 :
                name = 'xgb_chapter_models/xgbchap_model_5ksamp_v6_{}'.format(n_round)
                print("saving model: ", name)
                model.save_model(name)
                
        self.model  = model

## Carefully adjust Learning rate for each iteration so that training converges (else it won't work)

In [117]:
# lr =  [0.3]*100
lr =  [0.45]*14*2 + [0.4]*14*2 + [0.3] * 14 * 2 + [0.2]* 14 * 1 +  [0.1]* 14 * 1 +  [0.05]* 14 * 100

parameters = {'max_depth':7, 'objective':'multi:softprob', 'subsample':0.8, 
            'colsample_bytree':0.8, 'eta': 0.3, 'min_child_weight':0.1,
            'tree_method':'auto', 'num_class' : len(all_labels)
            }

model = make_model(parameters, lr, num_round=24, batch_size=10000) 
model.fit(X_train_counts, y_train)

[########################################] | 100% Completed |  0.2s
Will run for 24 rounds


  0%|          | 0/27 [00:00<?, ?it/s]

Round = 0, Iteration = 0, lr = 0.45
[0]	train-merror:0.563
Round = 0, Iteration = 1, lr = 0.45
[0]	train-merror:0.5203
Round = 0, Iteration = 2, lr = 0.45
[0]	train-merror:0.5053
Round = 0, Iteration = 3, lr = 0.45
[0]	train-merror:0.4945
Round = 0, Iteration = 4, lr = 0.45
[0]	train-merror:0.4795
Round = 0, Iteration = 5, lr = 0.45
[0]	train-merror:0.4699
Round = 0, Iteration = 6, lr = 0.45
[0]	train-merror:0.4635
Round = 0, Iteration = 7, lr = 0.45
[0]	train-merror:0.4459
Round = 0, Iteration = 8, lr = 0.45
[0]	train-merror:0.4421
Round = 0, Iteration = 9, lr = 0.45
[0]	train-merror:0.4316
Round = 0, Iteration = 10, lr = 0.45
[0]	train-merror:0.4363
Round = 0, Iteration = 11, lr = 0.45
[0]	train-merror:0.4315
Round = 0, Iteration = 12, lr = 0.45
[0]	train-merror:0.4254
Round = 0, Iteration = 13, lr = 0.45
[0]	train-merror:0.4217
Round = 0, Iteration = 14, lr = 0.45
[0]	train-merror:0.4215
Round = 0, Iteration = 15, lr = 0.45
[0]	train-merror:0.4175
Round = 0, Iteration = 16, lr = 0.4

  0%|          | 0/27 [00:00<?, ?it/s]

Round = 1, Iteration = 28, lr = 0.4
[0]	train-merror:0.3799
Round = 1, Iteration = 29, lr = 0.4
[0]	train-merror:0.3777
Round = 1, Iteration = 30, lr = 0.4
[0]	train-merror:0.3695
Round = 1, Iteration = 31, lr = 0.4
[0]	train-merror:0.3765
Round = 1, Iteration = 32, lr = 0.4
[0]	train-merror:0.3865
Round = 1, Iteration = 33, lr = 0.4
[0]	train-merror:0.3694
Round = 1, Iteration = 34, lr = 0.4
[0]	train-merror:0.3673
Round = 1, Iteration = 35, lr = 0.4
[0]	train-merror:0.3693
Round = 1, Iteration = 36, lr = 0.4
[0]	train-merror:0.3724
Round = 1, Iteration = 37, lr = 0.4
[0]	train-merror:0.3641
Round = 1, Iteration = 38, lr = 0.4
[0]	train-merror:0.3668
Round = 1, Iteration = 39, lr = 0.4
[0]	train-merror:0.3728
Round = 1, Iteration = 40, lr = 0.4
[0]	train-merror:0.3647
Round = 1, Iteration = 41, lr = 0.4
[0]	train-merror:0.3595
Round = 1, Iteration = 42, lr = 0.4
[0]	train-merror:0.3713
Round = 1, Iteration = 43, lr = 0.4
[0]	train-merror:0.3629
Round = 1, Iteration = 44, lr = 0.4
[0]	

...

Round = 21, Iteration = 588, lr = 0.05
[0]	train-merror:0.2989
Round = 21, Iteration = 589, lr = 0.05
[0]	train-merror:0.304
Round = 21, Iteration = 590, lr = 0.05
[0]	train-merror:0.3094
Round = 21, Iteration = 591, lr = 0.05
[0]	train-merror:0.2968
Round = 21, Iteration = 592, lr = 0.05
[0]	train-merror:0.2964
Round = 21, Iteration = 593, lr = 0.05
[0]	train-merror:0.2978
Round = 21, Iteration = 594, lr = 0.05
[0]	train-merror:0.3108
Round = 21, Iteration = 595, lr = 0.05
[0]	train-merror:0.3068
Round = 21, Iteration = 596, lr = 0.05
[0]	train-merror:0.3091
Round = 21, Iteration = 597, lr = 0.05
[0]	train-merror:0.3042
Round = 21, Iteration = 598, lr = 0.05
[0]	train-merror:0.2964
Round = 21, Iteration = 599, lr = 0.05
[0]	train-merror:0.2981
Round = 21, Iteration = 600, lr = 0.05
[0]	train-merror:0.3001
Round = 21, Iteration = 601, lr = 0.05
[0]	train-merror:0.3055
Round = 21, Iteration = 602, lr = 0.05
[0]	train-merror:0.3077
Round = 21, Iteration = 603, lr = 0.05
[0]	train-merror:

  0%|          | 0/27 [00:00<?, ?it/s]

Round = 22, Iteration = 616, lr = 0.05
[0]	train-merror:0.3024
Round = 22, Iteration = 617, lr = 0.05
[0]	train-merror:0.3032
Round = 22, Iteration = 618, lr = 0.05
[0]	train-merror:0.2982
Round = 22, Iteration = 619, lr = 0.05
[0]	train-merror:0.2976
Round = 22, Iteration = 620, lr = 0.05
[0]	train-merror:0.3007
Round = 22, Iteration = 621, lr = 0.05
[0]	train-merror:0.3068
Round = 22, Iteration = 622, lr = 0.05
[0]	train-merror:0.2954
Round = 22, Iteration = 623, lr = 0.05
[0]	train-merror:0.3084
Round = 22, Iteration = 624, lr = 0.05
[0]	train-merror:0.3069
Round = 22, Iteration = 625, lr = 0.05
[0]	train-merror:0.2962
Round = 22, Iteration = 626, lr = 0.05
[0]	train-merror:0.312
Round = 22, Iteration = 627, lr = 0.05
[0]	train-merror:0.3034
Round = 22, Iteration = 628, lr = 0.05
[0]	train-merror:0.2963
Round = 22, Iteration = 629, lr = 0.05
[0]	train-merror:0.3
Round = 22, Iteration = 630, lr = 0.05
[0]	train-merror:0.3008
Round = 22, Iteration = 631, lr = 0.05
[0]	train-merror:0.3

  0%|          | 0/27 [00:00<?, ?it/s]

Round = 23, Iteration = 644, lr = 0.05
[0]	train-merror:0.2987
Round = 23, Iteration = 645, lr = 0.05
[0]	train-merror:0.3006
Round = 23, Iteration = 646, lr = 0.05
[0]	train-merror:0.304
Round = 23, Iteration = 647, lr = 0.05
[0]	train-merror:0.3038
Round = 23, Iteration = 648, lr = 0.05
[0]	train-merror:0.3015
Round = 23, Iteration = 649, lr = 0.05
[0]	train-merror:0.3008
Round = 23, Iteration = 650, lr = 0.05
[0]	train-merror:0.303
Round = 23, Iteration = 651, lr = 0.05
[0]	train-merror:0.2881
Round = 23, Iteration = 652, lr = 0.05
[0]	train-merror:0.3026
Round = 23, Iteration = 653, lr = 0.05
[0]	train-merror:0.2959
Round = 23, Iteration = 654, lr = 0.05
[0]	train-merror:0.3026
Round = 23, Iteration = 655, lr = 0.05
[0]	train-merror:0.2935
Round = 23, Iteration = 656, lr = 0.05
[0]	train-merror:0.2992
Round = 23, Iteration = 657, lr = 0.05
[0]	train-merror:0.2988
Round = 23, Iteration = 658, lr = 0.05
[0]	train-merror:0.2952
Round = 23, Iteration = 659, lr = 0.05
[0]	train-merror:0

## Load the saved, trained model for further experiments

In [118]:
model.model = xgb.Booster(model_file='xgb_chapter_models/xgbchap_model_5ksamp_v6_23')

## Do prediction in batches, else it will crash with out of memory errors

In [119]:
def batch(lst, n=1):
    l = len(lst)
    for ndx in range(0, l, n):
        yield lst[ndx:min(ndx + n, l)]

def do_predict_batch(input):
    first  = []
    second = []
    third  = []
    fourth = []
    fifth  = []
    
    with tqdm(total=len(valid_df['text'])) as progress_bar:
        for x in batch(input, 2000):
            tmp_valid_counts = count_vector.transform(x)
            tmp_predict_da = model.model.predict(xgb.DMatrix(tmp_valid_counts.todense()))
            sorted_idx = np.argsort(-tmp_predict_da)
            first = first + list(label_enc.inverse_transform(list(sorted_idx[:,0])))
            second = second + list(label_enc.inverse_transform(list(sorted_idx[:,1])))
            third = third + list(label_enc.inverse_transform(list(sorted_idx[:,2])))
            fourth = fourth + list(label_enc.inverse_transform(list(sorted_idx[:,3])))
            fifth = fifth + list(label_enc.inverse_transform(list(sorted_idx[:,4])))
            progress_bar.update(2000)
        return first, second, third, fourth, fifth

y1, y2, y3, y4, y5 = do_predict_batch(list(valid_df['text']))

  0%|          | 0/67948 [00:00<?, ?it/s]

## Calculate accuracy for top 5 classes

In [120]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, plot_confusion_matrix, accuracy_score

a1 = accuracy_score(list(valid_df['label']), y1)
a2 = accuracy_score(list(valid_df['label']), y2)
a3 = accuracy_score(list(valid_df['label']), y3)
a4 = accuracy_score(list(valid_df['label']), y4)
a5 = accuracy_score(list(valid_df['label']), y5)

print('Accuracy:', a1, a2, a3, a4, a5)
print('Total:', a1+a2+a3+a4+a5)

Accuracy: 0.6595337611114381 0.11157061282156944 0.046358980396774004 0.026623300170718785 0.018013775239889326
Total: 0.8621004297403897


In [ ]:
import time
time.sleep(30)
print('Terminate Instance')
!aws ec2 terminate-instances --instance-ids i-0f41741a0c8b12972